In [3]:
# Comment: I opted to use Tabula because the best alternative I found, Camelot, was more
# complex to use and complex to install. However, based on the amount clean-up I did,
# it would be better to use Camelot if we were processing statements from many banks.

# I expcluded tabula's depencies from requirements.text since I don't use them.
# !pip install tabula-py    

# Parse the file using Tabula.
from decimal import Decimal
import tabula
import pandas as pd

pdf_path = "statements/idfc_first_bank.pdf"

# Read the PDF as a dataframe using tabula
data_frames = tabula.read_pdf(pdf_path, pages="all")

# Remove summary table
data_frames = data_frames[1:]

# Remove first row of data_framesf[0]
data_frames[0] = data_frames[0].iloc[1:]
# Reset index of the dataframe to start at 0
data_frames[0].reset_index(drop=True, inplace=True)

# Remove column 4 from df[0]
data_frames[0].drop(data_frames[0].columns[6], axis=1, inplace=True)

# Copy columns name from data_frames[1] to data_frames[0]
data_frames[0].columns = data_frames[1].columns

# Merge all dataframes in list of dataframes into a single dataframe
df = pd.concat(data_frames)

# Re-index the data frame
df.reset_index(drop=True, inplace=True)
del data_frames

# Remove every char in particulars except spaces or alphanumerics
# like [c for c in word of c.isalnum or c == ' ']
df["Particulars"] = df["Particulars"].apply(
    lambda x: "".join(" " if c.isspace() else c for c in x)
)
# df['Particulars'] = df['Particulars'].apply(lambda x: ''.join(c for c in x if c.isalnum() or c == " "))


# Some rows were split because "Particulars" column was multiple lines
# Merge them to fix this.
for i in range(len(df) - 1):
    if pd.isna(df.loc[i + 1, "Credit"]) and pd.isna(df.loc[i + 1, "Debit"]):
        df.loc[i, "Particulars"] += df.loc[i + 1, "Particulars"]

# Remove all empty rows
df.dropna(subset=["Credit", "Debit"], how="all", inplace=True)
df.reset_index(drop=True, inplace=True)


# Replace all Debit, Credit, Balance values with decimal.Decimal. pd.NaN becomes decimal.Decimal('0.0')
df["Debit"] = df["Debit"].apply(
    lambda x: Decimal("0.0") if pd.isna(x) else Decimal(str(x).replace(",", ""))
)
df["Credit"] = df["Credit"].apply(
    lambda x: Decimal("0.0") if pd.isna(x) else Decimal(str(x).replace(",", ""))
)
df["Balance"] = df["Balance"].apply(lambda x: Decimal(str(x).replace(",", "")))

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


0 IMPS-MOB/Fund Trf/809119519742/OmaRam/BENE - VER 

1 IMPS-MOB/Fund Trf/809119553510/OmaRam/2000 

2 NACH/TP ACH Bajaj Finanac/88551679 

3 APB-INW/HP644950 000009 3003 

4 POS-VISA/KARUN MEDICAL/809414290216 

5 IMPS-MOB/Fund Trf/809521437177/NITIN POOJARI/null 

6 BILLPAY/RECH/VODAFONE PRE/181000095662/10042018 

7 ATM-NFS/CASH WITHDRAWAL/+DONGRI OATM/810020011497 

8 ATM-NFS/CASHWITHDRAWAL/+HIRANANDANI AKRUTI/810418 

9 IMPS-RIB/Fund Trf/811010759849/PayRs300 

10 IMPS-RIB/FT-REV/811010759849/Pay Rs300 

11 POS-VISA/MANSI MEDICAL/811013502615 

12 IMPS-MOB/Fund Trf/811212400262/Chandrakant Yuv/BE 

13 IMPS-MOB/Fund Trf/811212428216/Chandrakant Yuv/ch 

14 IMPS-RIB/Fund Trf/811212820665/Paysalary 

15 IMPS-RIB/Fund Trf/811213820961/Payfor salary 

16 ATM-NFS/CASHWITHDRAWAL/+HIRANANDANI AKRUTI/811309 

17 IMPS-RIB/Fund Trf/811320861437/PaysalaryRs2500 

18 IMPS-RIB/Fund Trf/811321862644/Pay300 

19 IMPS-INET/Fund Trf/812321002185/Chandrakant Yuv/I 

20 NACH/TP ACH Bajaj Finanac/97127

/Users/owenbrown/code/cascade/venv/lib/python3.11/site-packages/tabula/io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
/var/folders/7r/0pxsk6n909s0q0y16qqthgp00000gn/T/ipykernel_8294/752282762.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(data_frames)


In [19]:
# The CEO requested the total monthly totals.
def get_total_monthly_deposits_and_credits(df: pd.DataFrame) -> int:
    df = df.copy()
    df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])
    df.set_index("Transaction Date", inplace=True)
    df["End of Month"] = df.index.to_period("M").to_timestamp("M")
    df = df.groupby("End of Month")[["Debit", "Credit"]].sum()
    df["Net Change"] = df["Credit"] - df["Debit"]
    return df


# Todo: End of Month is not actually end of month, but the day of the last transaction in the month
get_total_monthly_deposits_and_credits(df)

,Debit,Credit,Net Change
End of Month,,,
2018-04-30,10671.00,10671.20,0.20
2018-05-31,9583.00,9934.06,351.06
2018-06-30,2196.60,2189.19,-7.41
2018-07-31,8554.56,8234.41,-320.15
2018-08-31,3540.00,7501.00,3961.00
2018-09-30,23234.89,26451.63,3216.74
2018-10-31,21062.61,68944.89,47882.28
2018-11-30,55479.00,408.90,-55070.10
2018-12-31,14317.00,14380.33,63.33
